<a href="https://colab.research.google.com/github/IllyaUA/AI/blob/main/IronHackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Hello! Gilly here (Illia)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, make_scorer, cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load data.
I put files to ./data

In [23]:
#data = pd.read_csv(r'./data/train.csv')
data = pd.read_csv('/content/drive/MyDrive/IronWeek7_TechChallenge/data/train.csv')

display(data.shape, data['Class'].value_counts(), data.head())

(102, 20)

LIVE    81
DIE     21
Name: Class, dtype: int64

,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY,Class
0,48,meale,1,2,1,1,2,2,1,2,1,1,1,4.80,123,157,2.7,31,2,DIE
1,51,Male,2,2,1,2,2,2,1,1,1,2,1,1.00,?,20,3.0,63,2,LIVE
2,40,m,1,2,1,2,2,2,1,2,2,2,2,0.60,62,166,4.0,63,1,LIVE
3,25,m,2,2,1,2,2,1,1,1,1,1,1,1.30,181,181,4.5,57,2,LIVE
4,34,M,1,2,1,2,2,1,1,2,1,2,2,1.00,72,46,4.4,57,1,LIVE


In [ ]:
data.isnull().sum()/len(data)

AGE                0.0
SEX                0.0
STEROID            0.0
ANTIVIRALS         0.0
FATIGUE            0.0
MALAISE            0.0
ANOREXIA           0.0
LIVER BIG          0.0
LIVER FIRM         0.0
SPLEEN PALPABLE    0.0
SPIDERS            0.0
ASCITES            0.0
VARICES            0.0
BILIRUBIN          0.0
ALK PHOSPHATE      0.0
SGOT               0.0
ALBUMIN            0.0
PROTIME            0.0
HISTOLOGY          0.0
Class              0.0
dtype: float64

In [ ]:
numericals = data.select_dtypes(np.number)
numericals.head()

,AGE,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,HISTOLOGY
0,48,2,1,1,2,2
1,51,2,1,2,2,2
2,40,2,1,2,2,1
3,25,2,1,2,2,2
4,34,2,1,2,2,1


In [ ]:
numericals.describe

<bound method NDFrame.describe of      AGE  ANTIVIRALS  FATIGUE  MALAISE  ANOREXIA  HISTOLOGY
0     48           2        1        1         2          2
1     51           2        1        2         2          2
2     40           2        1        2         2          1
3     25           2        1        2         2          2
4     34           2        1        2         2          1
..   ...         ...      ...      ...       ...        ...
97    23           2        1        1         1          1
98    38           2        2        2         2          2
99    36           2        2        2         2          1
100   32           1        1        1         2          1
101   54           1        2        2         2          2

[102 rows x 6 columns]>

### Cleaning

In [ ]:
df=data.copy()
df['SEX'] = df['SEX'].apply(lambda x: 'F' if 'f' in x.lower() else 'M')
df['SEX'].value_counts()

M    92
F    10
Name: SEX, dtype: int64

In [ ]:
# Replace value 'c' in column 'col' with the mean of column 'col'.
def replace_by_median(data, col, c):
    data[col].mask(data[col] == c, 0, inplace=True)
    data[col] = pd.to_numeric(data[col])
    data[col].mask(data[col] == 0, data[col].median(), inplace=True)

def clean_categorical_columns(data):
    for col in data.columns:
        if pd.api.types.is_categorical_dtype(data[col]):
            unique_values = data[col].unique()
            for value in unique_values:
                if pd.notna(value):
                    replace_by_mean(data, col, value)
    return data

In [ ]:
def replace_question_marks_with_median(data):
    for col in data.columns:
        # Check if the column contains question marks
        if data[col].dtype == object and data[col].str.contains('\?').any():
            # Replace question marks with NaN
            data[col] = data[col].replace('?', np.nan)
            # Convert the column to a numeric type
            data[col] = pd.to_numeric(data[col])
            # Calculate the median of the column
            median = data[col].median()
            # Replace NaN values with the median
            data[col].fillna(median, inplace=True)
    return data

In [ ]:
#clean it
df=clean_categorical_columns(df)
display(df.head(),df['SEX'].value_counts())

,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY,Class
0,48,M,1,2,1,1,2,2,1,2,1,1,1,4.80,123,157,2.7,31,2,DIE
1,51,M,2,2,1,2,2,2,1,1,1,2,1,1.00,?,20,3.0,63,2,LIVE
2,40,M,1,2,1,2,2,2,1,2,2,2,2,0.60,62,166,4.0,63,1,LIVE
3,25,M,2,2,1,2,2,1,1,1,1,1,1,1.30,181,181,4.5,57,2,LIVE
4,34,M,1,2,1,2,2,1,1,2,1,2,2,1.00,72,46,4.4,57,1,LIVE


M    92
F    10
Name: SEX, dtype: int64

In [ ]:
df=replace_question_marks_with_median(df)
display(df.head(),df['SEX'].value_counts())

,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY,Class
0,48,M,1.0,2,1,1,2,2.0,1.0,2.0,1.0,1.0,1.0,4.8,123.0,157.0,2.7,31.0,2,DIE
1,51,M,2.0,2,1,2,2,2.0,1.0,1.0,1.0,2.0,1.0,1.0,85.0,20.0,3.0,63.0,2,LIVE
2,40,M,1.0,2,1,2,2,2.0,1.0,2.0,2.0,2.0,2.0,0.6,62.0,166.0,4.0,63.0,1,LIVE
3,25,M,2.0,2,1,2,2,1.0,1.0,1.0,1.0,1.0,1.0,1.3,181.0,181.0,4.5,57.0,2,LIVE
4,34,M,1.0,2,1,2,2,1.0,1.0,2.0,1.0,2.0,2.0,1.0,72.0,46.0,4.4,57.0,1,LIVE


M    92
F    10
Name: SEX, dtype: int64

now it is *clean*

# **Upsampling**

In [ ]:
# Upsampling (Get rid of imbalance in column 'Class' [DIE < LIVE])
die  = df[df['Class']=='DIE']
live = df[df['Class']=='LIVE']

die2 = resample(die, replace = True, n_samples = len(live))

df = pd.concat([die2, live])
df['Class'].value_counts()

DIE     81
LIVE    81
Name: Class, dtype: int64

# **EDA**

In [20]:
#Faryal does EDA here


# **X/y Split**

In [21]:
# Do the X/y split to seperate dependent from independent columns
X = df.drop('Class', axis=1)
y = df['Class']

# **Encode categoricals**

In [22]:
# Encode categorical columns to numerical (There's only one!)
X = pd.get_dummies(X, drop_first=True)

# **Train/Test split**

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **Apply StandardScaler**

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
X_test_scaled  = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

with open('standard_scaler.pickle', 'wb') as file:
    pickle.dump(scaler, file)

**Perform a GridSearch for RandomForestClassifier**

In [26]:
params = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'min_samples_leaf': [2,4],
    'min_samples_split': [2,4]
}

In [27]:
gs = GridSearchCV(estimator=RandomForestClassifier(),
                  param_grid=params, cv=5)
gs.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'min_samples_leaf': [2, 4],
                         'min_samples_split': [2, 4]})

In [28]:
gs.best_params_

{'criterion': 'entropy', 'min_samples_leaf': 2, 'min_samples_split': 4}